In [1]:
# ! pip install numpy

In [2]:
# ! pip install keras

In [3]:
#  ! pip install tensorflow

In [4]:
#  pip install opencv-python

In [18]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications import VGG16
import os

In [19]:
# import os
# import random
# import shutil

# # Set the path to the root folder
# root_folder = 'H:/20 New class'

# # Set the path to the new folder where the selected images will be copied
# new_folder = 'H:/20_class_8k'


# # Set the percentages for train, test, and validation
# train_percentage = 0.7
# test_percentage = 0.2
# validation_percentage = 0.1

# # Function to create the new folder if it doesn't exist
# def create_folder(folder_path):
#     if not os.path.exists(folder_path):
#         os.makedirs(folder_path)

# # Create the new folder structure for train, test, and validation
# create_folder(os.path.join(new_folder, 'train'))
# create_folder(os.path.join(new_folder, 'test'))
# create_folder(os.path.join(new_folder, 'validation'))

# # Iterate over the subfolders in the root folder
# for folder_name in os.listdir(root_folder):
#     folder_path = os.path.join(root_folder, folder_name)
    
#     if os.path.isdir(folder_path):
#         # Get the list of images in the current subfolder
#         images = os.listdir(folder_path)
        
#         # Shuffle the images randomly
#         random.shuffle(images)
        
#         # Calculate the number of images for each set
#         num_images = len(images)
#         num_train = int(train_percentage * num_images)
#         num_test = int(test_percentage * num_images)
#         num_validation = int(validation_percentage * num_images)
        
#         # Create subfolders in the new folder structure
#         create_folder(os.path.join(new_folder, 'train', folder_name))
#         create_folder(os.path.join(new_folder, 'test', folder_name))
#         create_folder(os.path.join(new_folder, 'validation', folder_name))
        
#         # Copy images to the train set
#         for image_name in images[:num_train]:
#             src_path = os.path.join(folder_path, image_name)
#             dst_path = os.path.join(new_folder, 'train', folder_name, image_name)
#             shutil.copy(src_path, dst_path)
        
#         # Copy images to the test set
#         for image_name in images[num_train:num_train+num_test]:
#             src_path = os.path.join(folder_path, image_name)
#             dst_path = os.path.join(new_folder, 'test', folder_name, image_name)
#             shutil.copy(src_path, dst_path)
        
#         # Copy images to the validation set
#         for image_name in images[num_train+num_test:num_train+num_test+num_validation]:
#             src_path = os.path.join(folder_path, image_name)
#             dst_path = os.path.join(new_folder, 'validation', folder_name, image_name)
#             shutil.copy(src_path, dst_path)


In [41]:
train_dir = 'H:/20_class_8k/Train'
valid_dir = 'H:/20_class_8k/Validation'
test_dir = 'H:/20_class_8k/Test'
target_size = (224, 224)  # VGG16 input size

datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

valid_generator = datagen.flow_from_directory(
    valid_dir,
    target_size=target_size,
    batch_size=32,\
    class_mode='categorical',
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 5600 images belonging to 20 classes.
Found 800 images belonging to 20 classes.
Found 1600 images belonging to 20 classes.


In [42]:
class_names = sorted(os.listdir(train_dir))

In [35]:
number_class=len(class_names)
print("Number Of Classes: ",number_class)

Number Of Classes:  20


In [36]:
print("Classes name are : ",class_names)

Classes name are :  ['ইউটার্ন', 'ইউটার্ন নেওয়া নিষেধ', 'একদিকে চলাচল', 'ওভারটেকিং নিষেধ', 'ট্র্যাক প্রবেশ নিষেধ', 'ডানদিকে মোড় নেওয়া নিষেধ', 'থামানো নিষেধ', 'পার্কিং', 'পার্কিং নিষেধ', 'প্রবেশ নিষেধ', 'বামদিকে মোড় নেওয়া নিষেধ', 'বাস স্ট্যান্ড', 'মসজিদ', 'রিকশা চলাচল নিষেধ', 'সামনে ডানদিকে আচমকা মোড় আছে', 'সামনে পথচারী পারাপার', 'সামনে বামদিকে পার্শ্ব রাস্তা আছে', 'সামনে স্কুল', 'হর্ন বাজানো নিষেধ', 'হাসপাতাল']


In [37]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [38]:
for layer in base_model.layers:
    layer.trainable = False


In [39]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))  # num_classes is the number of classes in your dataset




In [40]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=20,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // valid_generator.batch_size
)


Epoch 1/20
 30/122 [======>.......................] - ETA: 11:07 - loss: 2.5499 - accuracy: 0.3375

In [31]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print('Test accuracy:', test_acc)


17/17 [==============================] - 126s 8s/step - loss: 0.3012 - accuracy: 0.9375
Test accuracy: 0.9375


In [32]:
predictions = model.predict(test_generator, steps=test_generator.n // test_generator.batch_size)


17/17 [==============================] - 129s 8s/step


In [34]:
import numpy as np
from PIL import Image
# Load the image
image_path ="H:\\Dataset\\train\\No Overtaking\\image (2).png"
img = Image.open(image_path)
img = img.resize((224, 224))  # Resize the image to match the input size of VGG16

# Convert the image to RGB mode
img = img.convert("RGB")

# Preprocess the image
img_array = np.array(img)
img_array = img_array / 255.0  # Normalize the pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension

# Make the prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

print("Predicted class:", class_names[predicted_class])


1/1 [==============================] - 0s 190ms/step
Predicted class: No Overtaking
